In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import glob
import math
import plotly.express as px

import re, string
import seaborn as sns
import pickle


from sklearn.model_selection import train_test_split
import sys
sys.path.append('../')

from src.data.plot_functions import *
from src.data.feature_engineering import preprocess_train_df, tokenize
from src.conf.global_variables import TOXIC_COMMENT_DATA_PATH, SEED, LABELS, COMMENT
from src.models.train_model import fit_models
from src.models.predict_model import get_prediction
from src.models.save_and_load_models import load_models

%load_ext autoreload
%autoreload 2

## Load the Model 

In [2]:
models_filenames = glob.glob('./models/model_*.pkl')
vectorizer_filename = './models/vectorizer.pkl'

In [3]:
classifiers, vectorizer = load_models(models_filenames, vectorizer_filename)

## Prediction

In [4]:
sentence = 'hey fucking bitch'

In [5]:
preds, test_term_doc = get_prediction(sentence,vectorizer, classifiers, LABELS)

## Interpret prediction

Créons maintenant un dataframe propre avec les mots / poids

In [6]:
def get_words_weights_df(vectorizer, test_term_doc, classifiers, label):
    words_vectorizer = vectorizer.get_feature_names()

    words = np.array(words_vectorizer)[test_term_doc.indices]
    weights = classifiers[label].coef_.ravel()[test_term_doc.indices]

    df_words_weights = pd.DataFrame({'words': words,
                                     'weights': weights})
    return df_words_weights

In [7]:
df_words_weights = get_words_weights_df(vectorizer, test_term_doc, classifiers, 'toxic')

On peut d'abord recalculer la proba pour être sûr

In [8]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [9]:
all_x = classifiers['toxic'].intercept_.ravel() + df_words_weights['weights'].sum()

In [10]:
proba = sigmoid(all_x)
print(proba)

0.9917467790944695


In [11]:
def odds(x):
    return sigmoid(x) / (1 - sigmoid(x))

In [12]:
odds_with_feature = odds(all_x)
print(odds_with_feature)

120.16481691770775


Comment on interprète ça ?

Si on enlève fucking, 

In [13]:
sentence = 'hey bitch'

In [14]:
preds, test_term_doc_without_features = get_prediction(sentence,vectorizer, classifiers, LABELS)

In [15]:
df_words_weights_without_feature = get_words_weights_df(vectorizer,
                                                        test_term_doc_without_features,
                                                        classifiers,
                                                        'toxic')

In [16]:
df_words_weights_without_feature

,words,weights
0,bitch,2.639481
1,hey,0.016504


In [17]:
x_without_feature = classifiers['toxic'].intercept_.ravel() + df_words_weights_without_feature['weights'].sum()

#### Compare the actual odds_without_feature and the one using the exp(weight) 

In [18]:
odds_without_feature = odds(x_without_feature)
print(odds_without_feature)

1.3337305363497742


In [19]:
def odds_withdraw_feature(odds_with_feature, weight_feature):
    return odds_with_feature / np.exp(weight_feature)

In [20]:
odds_withdraw_feature(odds_with_feature, df_words_weights.weights[1])

1.3337305363497773

#### Compute proba without feature

In [21]:
preds[0][0]

0.5715015146675347

In [26]:
fig = px.bar(df_words_weights.sort_values(ascending= False, by='weights'), x ='words', y='weights',
             height=400)
fig.show()

In [27]:
df_words_weights['weights_exp'] = np.exp(df_words_weights['weights'])

fig = px.bar(df_words_weights, x ='words', y='weights_exp',
             height=400)
fig.show()